In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import numpy as np


In [ ]:
data = pd.read_csv('/content/lungcapacity.csv')

In [ ]:
print(data.isna().sum())

LungCap(cc)       10
Age( years)       10
Height(inches)     7
Smoke              6
Gender             2
Caesarean          4
No of children     0
Weight (kg)        7
dtype: int64


In [ ]:
data.head()

,LungCap(cc),Age( years),Height(inches),Smoke,Gender,Caesarean,No of children,Weight (kg)
0,6.475,6.0,62.1,NaN,male,no,3,85.70
1,10.125,18.0,74.7,yes,female,no,0,98.75
2,9.550,16.0,69.7,no,female,yes,0,11.01
3,11.125,14.0,71.0,no,male,no,1,29.78
4,4.800,5.0,56.9,no,male,no,4,72.84


In [ ]:
numeric_features = ['Age( years)', 'Height(inches)', 'Weight (kg)', 'No of children']
categorical_features = ['Smoke', 'Gender', 'Caesarean']

In [ ]:
for col in numeric_features:
    data[col].fillna(data[col].mean(), inplace=True)
for col in categorical_features:
    data[col].fillna(data[col].mode()[0], inplace=True)
data['LungCap(cc)'].fillna(data['LungCap(cc)'].mean(), inplace=True)

In [ ]:
X = data.drop('LungCap(cc)', axis=1)
y = data['LungCap(cc)']

In [ ]:
print(y.isna().sum())

0


In [ ]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

In [ ]:
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
X = preprocessor.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [ ]:
latent_dim = 100
data_dim = X_train.shape[1]

In [ ]:
def build_generator(latent_dim, data_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(data_dim, activation='tanh'))
    return model

In [ ]:
def build_discriminator(data_dim):
    model = Sequential()
    model.add(Dense(512, input_dim=data_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model


In [ ]:
generator = build_generator(latent_dim, data_dim)
discriminator = build_discriminator(data_dim)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

In [ ]:
z = Input(shape=(latent_dim,))
generated_data = generator(z)
discriminator.trainable = False
validity = discriminator(generated_data)

In [ ]:
gan = Model(z, validity)
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

In [ ]:
def train_gan(epochs, batch_size=128, save_interval=50):
    half_batch = int(batch_size / 2)
    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        real_data = X_train[idx]
        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        synthetic_data = generator.predict(noise)
        d_loss_real = discriminator.train_on_batch(real_data, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(synthetic_data, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_y = np.array([1] * batch_size)
        g_loss = gan.train_on_batch(noise, valid_y)
        if epoch % save_interval == 0:
            print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}] [G loss: {g_loss}]")

In [ ]:
train_gan(epochs=10000, batch_size=32, save_interval=1000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 43ms/step
6000 [D loss: 0.012367434334009886, acc.: 100.0] [G loss: 7.1772356033325195]
1/1 [==============================] - 0s 25ms/step
7000 [D loss: 0.11029830004554242, acc.: 93.75] [G loss: 10.62417221069336]
1/1 [==============================] - 0s 25ms/step
8000 [D loss: 0.03366125375032425, acc.: 96.875] [G loss: 8.84074592590332]
1/1 [==============================] - 0s 26ms/step
9000 [D loss: 0.016047482145950198, acc.: 100.0] [G loss: 10.178112030029297]
1/1 [==============================] - 0s 31ms/step


In [ ]:
def generate_synthetic_data(generator, num_samples, latent_dim):
    noise = np.random.normal(0, 1, (num_samples, latent_dim))
    synthetic_data = generator.predict(noise)
    return synthetic_data

In [ ]:
synthetic_data = generate_synthetic_data(generator, num_samples=746, latent_dim=latent_dim)

24/24 [==============================] - 0s 2ms/step


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
X_augmented = np.vstack((X_train, synthetic_data))
y_augmented = np.hstack((y_train, np.random.normal(y_train.mean(), y_train.std(), synthetic_data.shape[0])))
model = Sequential()
model.add(Dense(64, input_dim=X_augmented.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_augmented, y_augmented, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
34/34 [==============================] - 1s 8ms/step - loss: 54.1218 - val_loss: 32.5368
Epoch 2/100
34/34 [==============================] - 0s 3ms/step - loss: 16.6730 - val_loss: 7.9745
Epoch 3/100
34/34 [==============================] - 0s 4ms/step - loss: 7.7601 - val_loss: 7.7809
Epoch 4/100
34/34 [==============================] - 0s 4ms/step - loss: 6.4856 - val_loss: 8.0246
Epoch 5/100
34/34 [==============================] - 0s 3ms/step - loss: 5.8588 - val_loss: 7.9354
Epoch 6/100
34/34 [==============================] - 0s 4ms/step - loss: 5.4565 - val_loss: 8.0034
Epoch 7/100
34/34 [==============================] - 0s 3ms/step - loss: 5.1580 - val_loss: 7.9765
Epoch 8/100
34/34 [==============================] - 0s 3ms/step - loss: 4.9234 - val_loss: 7.8600
Epoch 9/100
34/34 [==============================] - 0s 4ms/step - loss: 4.7399 - val_loss: 7.8545
Epoch 10/100
34/34 [==============================] - 0s 4ms/step - loss: 4.6285 - val_loss: 7.6777
Epoch 

In [ ]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

5/5 [==============================] - 0s 3ms/step
Mean Squared Error: 1.6193758705300056
